In [3]:
#A杭州某高校
import concurrent.futures
import random
import pandas as pd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from datetime import datetime, timedelta
import numpy as np

class StudentAgent(Agent):
    def __init__(self, unique_id, model, class_id=None):
        super().__init__(unique_id, model)
        self.status = "S"  # 状态：S, E, I, Q，R
        self.roommates = []
        self.class_id = class_id  # 班级ID
        self.incubation_time = 0  # 潜伏期计时器
        self.infection_time = 0  # 感染时间计时器
        self.prev_status = "S"  # 初始化前一个状态为易感
        self.recovery_time = 0  # 恢复延时计时器

    def step(self):
        # 更新前一个状态
        self.prev_status = self.status

        # 同班接触、宿舍接触和随机接触
        if self.status == "I":
            if self.model.time_step in [6, 7, 12, 13, 18, 19]:  # 当时间步为假期时
                self.random_contact2()  # 只进行随机接触
            else:
                self.class_contact()
                self.dorm_contact()
                self.random_contact()

        # 状态更新逻辑
        if self.status == "E":
            self.incubation_time += 1
            if self.incubation_time >= 1.5 and random.random() < 0.23:
                self.status = "I"
            elif self.incubation_time >= 3.5 and random.random() < 0.25:
                self.status = "S"
        elif self.status == "I":
            self.infection_time += 1
            if self.infection_time >= 3.5 and random.random() < 0.95:
                self.status = "R"
            elif self.infection_time >= 6 and random.random() < 0.05:
                self.status = "E"  # 回到E状态

        # 恢复状态逻辑
        elif self.status == "R":
            self.recovery_time += 1
            if self.recovery_time >= 15.625 and random.random() < 0.062:
                self.status = "E"
        return

    def class_contact(self):
        neighbors = self.model.schedule.agents  # 获取所有代理
        class_neighbors = [other for other in neighbors if other.class_id == self.class_id]  # 仅同班学生
        for _ in range(600): 
            for other in class_neighbors:
                if other != self and self.status == "I" and other.status == "S":
                    if random.random() < 0.0006:  # 一个感染者与一个易感者每次接触并导致该易感者感染的概率
                        other.status = "E"

    def dorm_contact(self):
        for _ in range(300):  
            for roommate in self.roommates:
                if roommate != self and self.status == "I" and roommate.status == "S":
                    if random.random() < 0.0006:
                        roommate.status = "E"

    def random_contact(self):
        contacts = random.choices(self.model.schedule.agents, k=600)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"
    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"

                    
class SchoolModel(Model):

    def __init__(self, N, initial_infected):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(110, 110, True)
        self.data_collector = DataCollector(
            agent_reporters={"Status": "status"}
        )
        self.time_step = 1  # 设置开始日期
        self.start_date = datetime(2008, 6, 1)#初始化开始日期
        self.dates = [] 
        self.new_infections = []  
        self.current_infections = []  
        self.assign_dorms()

        num_classes = 290
    def run_single_model(self):
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % num_classes 
            selected_indices = np.random.choice(range(11099), size=9932, replace=False)
            has_dorm =  i in selected_indices
            
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i < initial_infected:
                a.status = "I"  # 设置初始感染者

        self.running = True
    def assign_dorms(self):
        dorms = [] 

        individuals_with_dorms = [agent for agent in self.schedule.agents if agent.has_dorm]

        # 分配四人间宿舍
        four_person_dorm_mates = []
        for i in range(0, len(individuals_with_dorms), 4):
            dorm_mates = individuals_with_dorms[i:i + 4]
            if len(dorm_mates) == 4:
                for mate in dorm_mates:
                    mate.roommates = dorm_mates 
                four_person_dorm_mates.append(dorm_mates)

        dorms.append(four_person_dorm_mates)
        return dorms
    
    def step(self):
        new_infections_today = 0  
        self.schedule.step() 

        for agent in self.schedule.agents:
            if agent.status == "I" and agent.prev_status == "E":
                new_infections_today += 1

        self.new_infections.append(new_infections_today)
        
        # 更新日期
        self.time_step += 1
        self.dates.append(self.start_date + timedelta(days=len(self.dates)))
        self.data_collector.collect(self)
        
    def get_results(self):
        return pd.DataFrame({
            "Day": range(len(self.new_infections)),
            "New Infections": self.new_infections,
        })
    def run_single_model(self):
        # 重置模型
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        # 初始化学生代理
        for i in range(self.num_agents):
            class_id = i % 290  
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i == 0:  # 只有第一个代理是感染者
                a.status = "I"

        # 模拟40天
        for _ in range(24): 
            self.step()

        return self.new_infections, self.dates  

    def run_model(self, **kwargs):  
        num_runs = kwargs.get('num_runs', 1)  # 获取 num_runs 参数，如果不存在则默认为 1
        all_results = []  

        for _ in range(num_runs):
            new_infections, dates = self.run_single_model()
            all_results.append(new_infections)

        # 创建结果DataFrame
        results_df = pd.DataFrame(all_results).T
        results_df.columns = [f'Run {i+1}' for i in range(num_runs)]
        results_df['Date'] = [i for i in range(1, len(dates) + 1)]
        results_df = results_df.set_index('Date')

        print(results_df)
        results_df.to_csv(r"C:\Users\Administrator\Desktop\tb break\B_0006_20_10_20.csv", index=False)

if __name__ == "__main__":
    model = SchoolModel(11099, initial_infected=1)
    model.run_model(num_runs=3)  # 指定循环次数

      Run 1  Run 2  Run 3
Date                     
1         0      0      0
2         2      2      3
3         3      2      2
4         3      7      6
5         9      5      9
6         5      6      4
7         3      8      4
8         6     12      8
9        15     14      9
10       14     33     22
11       44     38     43
12       67     71     64
13       67     80     67
14       68    147    113
15      143    194    164
16      209    233    180
17      270    319    293
18      431    522    325
19      443    581    515
20      426    756    636
21      724    880    813
22      978    985    959
23     1111   1068   1086
24     1142   1004   1022


In [1]:
#B天台县高中
import concurrent.futures
import random
import pandas as pd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from datetime import datetime, timedelta
import numpy as np

class StudentAgent(Agent):
    """
    一个学生代理，代表学校里的每个学生
    """

    def __init__(self, unique_id, model, class_id=None):
        super().__init__(unique_id, model)
        self.status = "S"  # 状态：S, E, I, Q，R
        self.roommates = []
        self.class_id = class_id  # 班级ID
        self.incubation_time = 0  # 潜伏期计时器
        self.infection_time = 0  # 感染时间计时器
        self.prev_status = "S"  # 初始化前一个状态为易感
        self.recovery_time = 0  # 恢复延时计时器

    def step(self):
        # 更新前一个状态
        self.prev_status = self.status

        # 同班接触、宿舍接触和随机接触
        if self.status == "I":
            if self.model.time_step in [8, 9, 14, 15, 20, 21]:  # 当时间步为假期时
                self.random_contact2()  # 只进行随机接触
            else:
                self.class_contact()
                self.dorm_contact()
                self.random_contact()

        # 状态更新逻辑
        if self.status == "E":
            self.incubation_time += 1
            if self.incubation_time >= 1.5 and random.random() < 0.23:
                self.status = "I"
            elif self.incubation_time >= 3.5 and random.random() < 0.25:
                self.status = "S"
        elif self.status == "I":
            self.infection_time += 1
            if self.infection_time >= 3.5 and random.random() < 0.95:
                self.status = "R"
            elif self.infection_time >= 6 and random.random() < 0.05:
                self.status = "E"  # 回到E状态

        # 恢复状态逻辑
        elif self.status == "R":
            self.recovery_time += 1
            if self.recovery_time >= 15.625 and random.random() < 0.062:
                self.status = "E"
        return

    def class_contact(self):
        neighbors = self.model.schedule.agents  # 获取所有代理
        class_neighbors = [other for other in neighbors if other.class_id == self.class_id]  # 仅同班学生
        for _ in range(600): 
            for other in class_neighbors:
                if other != self and self.status == "I" and other.status == "S":
                    if random.random() < 0.0006:  # 一个感染者与一个易感者每次接触并导致该易感者感染的概率
                        other.status = "E"

    def dorm_contact(self):
        for _ in range(300):  
            for roommate in self.roommates:
                if roommate != self and self.status == "I" and roommate.status == "S":
                    if random.random() < 0.0006:
                        roommate.status = "E"

    def random_contact(self):
        contacts = random.choices(self.model.schedule.agents, k=600)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"
    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"

class SchoolModel(Model):
    """学校模型"""

    def __init__(self, N, initial_infected):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(60, 60, True)
        self.data_collector = DataCollector(
            agent_reporters={"Status": "status"}
        )
        self.time_step = 1  # 设置开始日期
        self.start_date = datetime(2008, 5, 1)#初始化开始日期
        self.dates = [] 
        self.new_infections = []  
        self.current_infections = []  
        self.assign_dorms()

        num_classes = 54
        
    def run_single_model(self):
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % num_classes 
            selected_indices = np.random.choice(range(2896), size=2570, replace=False)
            has_dorm =  i in selected_indices
            
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i < initial_infected:
                a.status = "I"  # 设置初始感染者

        self.running = True
        
    def assign_dorms(self):
        dorms = [] 

        individuals_with_dorms = [agent for agent in self.schedule.agents if agent.has_dorm]

        # 分配六人间宿舍
        six_person_dorm_mates = []
        for i in range(0, len(individuals_with_dorms), 6):
            dorm_mates = individuals_with_dorms[i:i + 6]
            if len(dorm_mates) == 6:
                for mate in dorm_mates:
                    mate.roommates = dorm_mates 
                six_person_dorm_mates.append(dorm_mates)

        dorms.append(six_person_dorm_mates)
        return dorms
    
    def step(self):
        new_infections_today = 0
        self.schedule.step()

        #current_infected = sum(1 for agent in self.schedule.agents if agent.status == "I")

        for agent in self.schedule.agents:
            if agent.status == "I" and agent.prev_status == "E":
                new_infections_today += 1

        self.new_infections.append(new_infections_today)
        #self.current_infections.append(current_infected)
        
        # 更新日期
        self.time_step += 1
        self.dates.append(self.start_date + timedelta(days=len(self.dates)))
        self.data_collector.collect(self)

    def get_results(self):
        return pd.DataFrame({
            "Date": self.dates,
            "New Infections": self.new_infections,
            #"Current Infections": self.current_infections
        })

    def run_single_model(self):
        # 重置模型
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []

        # 初始化学生代理
        for i in range(self.num_agents):
            class_id = i % 54  
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i == 0:  # 只有第一个代理是感染者
                a.status = "I"

        # 模拟40天
        for _ in range(24): 
            self.step()

        return self.new_infections, self.dates  

    def run_model(self, **kwargs):  
        num_runs = kwargs.get('num_runs', 1)  # 获取 num_runs 参数，如果不存在则默认为 1
        all_results = []  

        for _ in range(num_runs):
            new_infections, dates = self.run_single_model()
            all_results.append(new_infections)

        # 创建结果DataFrame
        results_df = pd.DataFrame(all_results).T
        results_df.columns = [f'Run {i+1}' for i in range(num_runs)]
        results_df['Date'] = [i for i in range(1, len(dates) + 1)]
        results_df = results_df.set_index('Date')

        print(results_df)
        results_df.to_csv(r"C:\Users\Administrator\Desktop\tb break\B_0006_20_10_20.csv", index=False)

if __name__ == "__main__":
    model = SchoolModel(2896, initial_infected=1)
    model.run_model(num_runs=20)  # 指定循环次数

      Run 1  Run 2  Run 3  Run 4  Run 5  Run 6  Run 7  Run 8  Run 9  Run 10  \
Date                                                                          
1         0      0      0      0      0      0      0      0      0       0   
2         3      0      0      0      2      0      0      0      2       0   
3         3      5     12      9      3     11      3      6      4       8   
4        12      3      4      5     11      2      4     11      4       9   
5        15      3      9     14      8     13      8      4     14       5   
6        16     14     13      3      6     10     10     10     14      20   
7        12     10     10      7      8     13      4      7     17      16   
8         8      9     23     17     18     29      9      9     40      27   
9        25      6     46     28     21     43     12     27     41      41   
10       16     17     63     32     49     60     19     44     71      45   
11       57     26     94     67     78     97     4

In [ ]:
#C嵊州某高中
import concurrent.futures
import random
import pandas as pd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from datetime import datetime, timedelta
import numpy as np

class StudentAgent(Agent):
    """
    一个学生代理，代表学校里的每个学生
    """

    def __init__(self, unique_id, model, class_id=None):
        super().__init__(unique_id, model)
        self.status = "S"  # 状态：S, E, I, Q，R
        self.roommates = []
        self.class_id = class_id  # 班级ID
        self.incubation_time = 0  # 潜伏期计时器
        self.infection_time = 0  # 感染时间计时器
        self.prev_status = "S"  # 初始化前一个状态为易感
        self.recovery_time = 0  # 恢复延时计时器

    def step(self):
        # 更新前一个状态
        self.prev_status = self.status

        # 同班接触、宿舍接触和随机接触
        if self.status == "I":
            if self.model.time_step in [6, 7, 12, 13, 18, 19]:  # 当时间步为假期时
                self.random_contact2()  # 只进行随机接触
            else:
                self.class_contact()
                self.dorm_contact()
                self.random_contact()

        # 状态更新逻辑
        if self.status == "E":
            self.incubation_time += 1
            if self.incubation_time >= 1.5 and random.random() < 0.23:
                self.status = "I"
                self.isolation_delay = 0  # 设置延迟时间，假设为2轮
            elif self.incubation_time >= 3.5 and random.random() < 0.25:
                self.status = "S"
        elif self.status == "I":
            self.infection_time += 1
            if self.infection_time >= 3.5 and random.random() < 0.95:
                self.status = "R"
            elif self.infection_time >= 6 and random.random() < 0.05:
                self.status = "E"  

        # 恢复状态逻辑
        elif self.status == "R":
            self.recovery_time += 1
            if self.recovery_time >= 15.625 and random.random() < 0.062:
                self.status = "E"
        return

    def class_contact(self):
        neighbors = self.model.schedule.agents  # 获取所有代理
        class_neighbors = [other for other in neighbors if other.class_id == self.class_id]  # 仅同班学生
        for _ in range(600):  
            for other in class_neighbors:
                if other != self and self.status == "I" and other.status == "S":
                    if random.random() < 0.00003:  # 一个感染者与一个易感者每次接触并导致该易感者感染的概率
                        other.status = "E"

    def dorm_contact(self):
        for _ in range(300):  
            for roommate in self.roommates:
                if roommate != self and self.status == "I" and roommate.status == "S":
                    if random.random() < 0.00003:
                        roommate.status = "E"

    def random_contact(self):
        contacts = random.choices(self.model.schedule.agents, k=600)

        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.00003:
                    other.status = "E"

    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"

    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"
                    
class SchoolModel(Model):

    def __init__(self, N, initial_infected):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(60, 60, True)
        self.data_collector = DataCollector(
            agent_reporters={"Status": "status"}
        )
        self.time_step = 1  # 设置开始日期
        self.start_date = datetime(2009, 1, 1)#初始化开始日期
        self.dates = []  
        self.new_infections = [] 
        self.current_infections = []  
        self.assign_dorms()

        num_classes = 28
    def run_single_model(self):
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % num_classes 
            selected_indices = np.random.choice(range(1419), size=616, replace=False)
            has_dorm =  i in selected_indices
            
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i < initial_infected:
                a.status = "I"  # 设置初始感染者
        self.running = True
        
    def assign_dorms(self):
        dorms = [] 
        individuals_with_dorms = [agent for agent in self.schedule.agents if agent.has_dorm]

        def assign_dorm(individuals, size):
            dorm_mates = []
            for i in range(0, len(individuals), size):
                dorm_group = individuals[i:i + size]
                if len(dorm_group) == size:
                    for mate in dorm_group:
                        mate.roommates = dorm_group  # 将宿舍成员分配给每个学生
                    dorm_mates.append(dorm_group)
            return dorm_mates

        # 随机选择分配六人间还是八人间
        if random.choice([True, False]):  # 随机选择 True 或 False
            six_person_dorms = assign_dorm(individuals_with_dorms, 6)
            dorms.append(six_person_dorms)
        else:
            eight_person_dorms = assign_dorm(individuals_with_dorms, 8)
            dorms.append(eight_person_dorms)

        return dorms
    
    def step(self):
        new_infections_today = 0 
        self.schedule.step()

        for agent in self.schedule.agents:
            if agent.status == "I" and agent.prev_status == "E":
                new_infections_today += 1

        self.new_infections.append(new_infections_today)

        # 更新日期
        self.time_step += 1
        self.dates.append(self.start_date + timedelta(days=len(self.dates)))
        self.data_collector.collect(self)

    def get_results(self):
        return pd.DataFrame({
            "Date": self.dates,
            "New Infections": self.new_infections
        })

    def run_single_model(self):
        # 重置模型
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []

        for run in range(self.num_agents):
            self.schedule = RandomActivation(self)
            self.dates = []
            self.new_infections = []
            
            for i in range(self.num_agents):
                class_id = i % 28  
                a = StudentAgent(i, self, class_id)
                self.schedule.add(a)
                self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
                if i == 0:  
                    a.status = "I"

            # 模拟24月
            for _ in range(24):
                self.step()
            
    def run_model(self, **kwargs):  
        num_runs = kwargs.get('num_runs', 1)  # 获取 num_runs 参数，如果不存在则默认为 1
        all_results = []  

        for _ in range(num_runs):
            new_infections, dates = self.run_single_model()
            all_results.append(new_infections)

        # 创建结果DataFrame
        results_df = pd.DataFrame(all_results).T
        results_df.columns = [f'Run {i+1}' for i in range(num_runs)]
        results_df['Date'] = [i for i in range(1, len(dates) + 1)]
        results_df = results_df.set_index('Date')

        print(results_df)
        results_df.to_csv(r"C:\Users\Administrator\Desktop\tb break\C_5_3_5_20.csv", index=False)

if __name__ == "__main__":
    model = SchoolModel(1419, initial_infected=1)
    model.run_model(num_runs=20)  # 设定循环次数

In [1]:
#D贵州省某高中
import concurrent.futures
import random
import pandas as pd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from datetime import datetime, timedelta
import numpy as np

class StudentAgent(Agent):
    def __init__(self, unique_id, model, class_id=None):
        super().__init__(unique_id, model)
        self.status = "S"  # 状态：S, E, I, Q，R
        self.roommates = []
        self.class_id = class_id  # 班级ID
        self.incubation_time = 0  # 潜伏期计时器
        self.infection_time = 0  # 感染时间计时器
        self.prev_status = "S"  # 初始化前一个状态为易感
        self.recovery_time = 0  # 恢复延时计时器

    def step(self):
        # 更新前一个状态
        self.prev_status = self.status

        # 同班接触、宿舍接触和随机接触
        if self.status == "I":
            if self.model.time_step in [0, 5, 6, 11, 12, 17, 18, 23]:  # 当时间步为假期时
                self.random_contact2()  
            else:
                self.class_contact()
                self.dorm_contact()
                self.random_contact()
                
        # 状态更新逻辑
        if self.status == "E":
            self.incubation_time += 1
            if self.incubation_time >= 1.5 and random.random() < 0.23:
                self.status = "I"
            elif self.incubation_time >= 3.5 and random.random() < 0.25:
                self.status = "S"
        elif self.status == "I":
            self.infection_time += 1
            if self.infection_time >= 3.5 and random.random() < 0.95:
                self.status = "R"
            elif self.infection_time >= 6 and random.random() < 0.05:
                self.status = "E"

        # 恢复状态逻辑
        elif self.status == "R":
            self.recovery_time += 1
            if self.recovery_time >= 15.625 and random.random() < 0.062:
                self.status = "E"
        return

    def class_contact(self):
        neighbors = self.model.schedule.agents  # 获取所有代理
        class_neighbors = [other for other in neighbors if other.class_id == self.class_id]  # 仅同班学生
        for _ in range(600):  # 每天接触四次
            for other in class_neighbors:
                if other != self and self.status == "I" and other.status == "S":
                    if random.random() < 0.00003:  # 一个感染者与一个易感者每次接触并导致该易感者感染的概率
                        other.status = "E"

    def dorm_contact(self):
        for _ in range(300):  # 每天接触五次
            for roommate in self.roommates:
                if roommate != self and self.status == "I" and roommate.status == "S":
                    if random.random() < 0.00003:
                        roommate.status = "E"

    def random_contact(self):
        contacts = random.choices(self.model.schedule.agents, k=600)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.00003:
                    other.status = "E"
    def random_contact2(self):
        contacts = random.choices(self.model.schedule.agents, k=800)
        for other in contacts:
            if other != self and self.status == "I" and other.status == "S":
                if random.random() < 0.0006:
                    other.status = "E"
class SchoolModel(Model):
    """学校模型"""

    def __init__(self, N, initial_infected):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(60, 70, True)
        self.data_collector = DataCollector(
            agent_reporters={"Status": "status"}
        )
        self.time_step = 1  # 设置开始日期
        self.start_date = datetime(2008, 5, 1)#初始化开始日期
        self.dates = []  
        self.new_infections = []  
        self.current_infections = []  
        self.students = []  

        num_classes = 54
    def run_single_model(self):
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % num_classes 
            selected_indices = np.random.choice(range(3655), size=1745, replace=False)
            has_dorm =  i in selected_indices
            
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i < initial_infected:
                a.status = "I"  # 设置初始感染者

        self.running = True
        
    def assign_dorms(self):
        dorms = []  

        individuals_with_dorms = [agent for agent in self.schedule.agents if agent.has_dorm]

        # 分配十人间宿舍
        six_person_dorm_mates = []
        for i in range(0, len(individuals_with_dorms), 10):
            dorm_mates = individuals_with_dorms[i:i + 10]
            if len(dorm_mates) == 10:
                for mate in dorm_mates:
                    mate.roommates = dorm_mates  
                six_person_dorm_mates.append(dorm_mates)

        dorms.append(six_person_dorm_mates)

        # 分配八人间宿舍
        eight_person_dorm_mates = []
        for i in range(0, len(individuals_with_dorms), 8):
            dorm_mates = individuals_with_dorms[i:i + 8]
            if len(dorm_mates) == 8:
                for mate in dorm_mates:
                    mate.roommates = dorm_mates  
                eight_person_dorm_mates.append(dorm_mates)

        dorms.append(eight_person_dorm_mates)
        return dorms  
    
    def step(self):
        new_infections_today = 0  
        self.schedule.step()  


        for agent in self.schedule.agents:
            if agent.status == "I" and agent.prev_status == "E":
                new_infections_today += 1

        self.new_infections.append(new_infections_today)

        # 更新日期
        self.time_step += 1
        self.dates.append(self.start_date + timedelta(days=len(self.dates)))
        self.data_collector.collect(self)

    def get_results(self):
        return pd.DataFrame({
            "Date": self.dates,
            "New Infections": self.new_infections,
            "Current Infections": self.current_infections
        })
    
    def run_single_model(self):
        # 重置模型
        self.schedule = RandomActivation(self)
        self.dates = []
        self.new_infections = []
        
        for i in range(self.num_agents):
            class_id = i % 54  
            a = StudentAgent(i, self, class_id)
            self.schedule.add(a)
            self.grid.place_agent(a, (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)))
            if i == 0:  # 只有第一个代理是感染者
                a.status = "I"
        for _ in range(24): 
            self.step()
        return self.new_infections, self.dates  
    def run_model(self, **kwargs):  
        num_runs = kwargs.get('num_runs', 1)  # 获取 num_runs 参数，如果不存在则默认为 1
        all_results = []  

        for _ in range(num_runs):
            new_infections, dates = self.run_single_model()
            all_results.append(new_infections)

        # 创建结果DataFrame
        results_df = pd.DataFrame(all_results).T
        results_df.columns = [f'Run {i+1}' for i in range(num_runs)]
        results_df['Date'] = [i for i in range(1, len(dates) + 1)]
        results_df = results_df.set_index('Date')

        print(results_df)
        results_df.to_csv(r"C:\Users\Administrator\Desktop\tb break\D_0006_20_10_20.csv", index=False)

if __name__ == "__main__":
    model = SchoolModel(3655, initial_infected=2)
    model.run_model(num_runs=20)  # 设定循环次数

      Run 1  Run 2  Run 3  Run 4  Run 5  Run 6  Run 7  Run 8  Run 9  Run 10  \
Date                                                                          
1         0      0      0      0      0      0      0      0      0       0   
2         1      0      0      0      0      0      0      0      0       0   
3         0      0      0      1      0      0      0      1      0       0   
4         1      0      1      0      1      0      1      0      0       0   
5         1      0      3      1      1      0      1      0      2       1   
6         0      1      0      2      0      0      0      2      2       2   
7         2      0      1      2      1      0      1      1      0       0   
8         2      2      2      3      1      0      2      2      2       1   
9         2      0      3      3      2      1      1      2      2       1   
10        1      2      3      3      0      0      1      2      4       3   
11        5      0      2      4      4      1      